In [1]:
import numpy as np
import pandas as pd
import fileinput
import json
from tqdm import tqdm
tqdm.pandas()

import pickle

from tssplit import tssplit
import re

!pip3 install numpy

In [3]:
%%time

df = pd.read_json('/home/dave/NLP/WashingtonPost-v4/data/TREC_Washington_Post_collection.v4.jl', lines=True)
  


CPU times: user 3min 12s, sys: 3min 12s, total: 6min 24s
Wall time: 6min 40s


In [4]:
#im ersten durchlauf gedropped
df.drop('article_url', inplace=True, axis=1)
df.drop('published_date', inplace=True, axis=1)
df.drop('content', inplace=True, axis=1)
df.drop('publish_date', inplace=True, axis=1)
df.drop('orig-id', inplace=True, axis=1)

In [5]:
#im zweiten durchlauf gedropped
df.drop('id', inplace=True, axis=1)
df.drop('type', inplace=True, axis=1)
df.drop('source', inplace=True, axis=1)


# 2

In [7]:
def remove_duplicates(l):
    return list(set(l))

In [8]:
pattern_auth = "\s*\w*"
drop_list = []
index = 0
i = 0



for auth in df["author"]:
  i = 0
  at = re.search(pattern_auth, str(auth))
  if at or len(str(auth)) <= 4:
      drop_list.append(i)
  if auth == "":
    drop_list.append(i)
  i+= 1
    
for tit in df["title"]:
  i = 0
  if len(str(tit)) <= 4:
    drop_list.append(i)
  if auth == "":
    drop_list.append(i)
  i+= 1
  


In [9]:
df = df.drop(remove_duplicates(drop_list))

In [10]:

df = df.sort_index()
df = df.reset_index(drop=True)


In [12]:
%%time
le = int(len(df)/8)
start = 0
end = le
i = 1 
while i < 9:
    df[start:end].to_pickle("/home/dave/NLP/WashingtonPost-v4/data/"+str(i)+".pkl")
    start = end
    end = end + le
    i += 1

CPU times: user 1min 35s, sys: 9.49 s, total: 1min 45s
Wall time: 2min 1s


# Kernel neu starten

In [2]:
import numpy as np
import pandas as pd
import fileinput
import json
from tqdm import tqdm
tqdm.pandas()

import pickle

from tssplit import tssplit
import re

In [21]:
### nur beim zweiten durchlauf
dfKat_behalten = pd.read_csv("P:/uni/nlp/WashingtonPost_v4/Kategorie_behalten.CSV",header=None)
Kat_behalten = list(dfKat_behalten[0])
bool_kat = True

In [23]:
### nur beim zweiten durchlauf
### funktion 
def ist_in_kat(x):
    if x in Kat_behalten:
        bool_kat_temp = False
    else:
        bool_kat_temp = True
    return bool_kat_temp


for thema in df[""]:
  i = 0
  bool_kat = ist_in_kat(thema)
  if bool_kat:
    drop_list.append(i)
  i+=1
    

In [2]:
%%time
pattern_paragraph = "\'subtype\': \'paragraph\'"
pattern_date = "\d{4}-\d{2}-\d{2}T\d{2}:\d{2}:\d{2}Z"



for c in range(1,9):
    
    df = pd.read_pickle("/home/dave/NLP/WashingtonPost-v4/data/"+str(c)+".pkl")
    
    df = df.reset_index(drop=True)
    
    meta_List = []
    content_List = []

    j= 0

    for article in df["contents"]:
        temp_List_meta = []
        temp_List_content = []
        temp_List_content2 = []
        temp_List_meta2 = []
        temp_List_meta4 = []

        for eintrag in article:
            x = re.search(pattern_paragraph, str(eintrag))
            if x:
                temp_List_content = [eintrag]
                for t in temp_List_content:
                    val = str(t)
                    val3 = tssplit(val,quote="'", delimiter=",")
                    t4 = val3[0][10:]
                    temp_List_content2.append(t4)
            else:
                temp_List_meta = [eintrag]
                for t in temp_List_meta:

                    val2 = str(t)
                    val6 = val2.split(",")
                    t8 = val6[0][13:]
                    temp_List_meta2.append(t8)
        
        ### hier eine abfrage nach den Kategorien aus der liste machen
        
            if  bool_kat:
                temp_List_meta4.append(temp_List_meta2[0])
            #temp_List_meta4.append(temp_List_meta2[1])
            else:
                print()
        
        
        
        temp_List_meta4.append(df["title"][j])   #neu muss noch getetstet werden
        temp_List_meta4.append(df["author"][j])

        for m in temp_List_meta2:                
            d = re.search(pattern_date, str(m))          
            if d:
                temp_List_meta4.append(m)
            else:
                continue

        if len(temp_List_meta4) >= 3 and len(temp_List_meta4) <= 4 and len(temp_List_content2) > 3:
            #if temp_List_meta4[0]
            meta_List.append(temp_List_meta4)    
            content_List.append("".join(temp_List_content2))

        j+=1        

    dfMeta = pd.DataFrame(meta_List)
    dfContent = pd.DataFrame(content_List)

    dfContent = dfContent.set_axis(["content"], axis=1, inplace = False)

    dfMeta = dfMeta.set_axis(["Kategorie","Titel","Autor","Datum"], axis=1, inplace = False)

    dfContent["id"] = dfContent.index
    dfMeta["id1"] = dfMeta.index

    df = pd.merge(dfMeta,dfContent, left_on="id1", right_on="id", how="left").drop("id1",axis=1)
    
    df.to_pickle("/home/dave/NLP/WashingtonPost-v4/data/"+str(c)+"_raw.pkl")

    c += 1

FileNotFoundError: [Errno 2] No such file or directory: '/home/dave/NLP/WashingtonPost-v4/data/1.pkl'

# Kernel neu starten

## 5

In [1]:
from tqdm import tqdm
import numpy as np
import pandas as pd
tqdm.pandas()
import fileinput
import pickle
import re
import spacy
from pandarallel import pandarallel

!python -m spacy download en_core_web_md <br>
!python -m spacy download en_core_web_lg

In [4]:
%%time

for c in range(1,9):
    
    df = pd.read_pickle("/home/dave/NLP/WashingtonPost-v4/data/"+str(c)+"_raw.pkl")
    
    df = df.set_index('id')
    
    df.Kategorie = df.Kategorie.replace("\"$|\'$", "",regex=True)
    df.Titel = df.Titel.replace("\"$|\'$", "",regex=True)
    df.Datum = df.Datum.replace("\"$|\'$", "",regex=True) 

    df.content = df.content.replace("\<.*?\>","",regex=True)
    
    df.to_pickle("data/"+str(c)+"_clean.pkl")

CPU times: user 37.8 s, sys: 8.24 s, total: 46 s
Wall time: 55 s


# Kernel neu starten

In [1]:
from tqdm import tqdm
import numpy as np
import pandas as pd
tqdm.pandas()
import fileinput
import pickle
import re
import spacy
from pandarallel import pandarallel

In [2]:
nlp = spacy.load("en_core_web_lg")
pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 22 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [3]:
all_stopwords = nlp.Defaults.stop_words
def remove_stopwords_from_list(list_in):
    return [a for a in list_in if a not in all_stopwords]

In [3]:
df2 = pd.read_pickle("P:/uni/nlp/WashingtonPost_v4/data/v5/5_clean.pkl")
#df3 = pd.read_pickle("/home/dave/NLP/WashingtonPost-v4/data/3_clean.pkl")

In [5]:
df2["tokens"] = df2.parallel_apply(lambda row: [a.text for a in list(nlp(str(row["content"])))], axis=1)
df2["no_stop"] = df2.parallel_apply(lambda row: remove_stopwords_from_list(row["tokens"]), axis=1)
df2["lemma"] = df2.parallel_apply(lambda row: [t.lemma_ for t in nlp(row["content"])], axis=1)

In [3]:
from tqdm import tqdm
import numpy as np
import pandas as pd
tqdm.pandas()
import fileinput
import pickle
import re
import spacy
from pandarallel import pandarallel

In [4]:
kategorie_all = []
kategorie = []

In [5]:
kategorie = []

for c in range(1,9):
    
    df2 = pd.read_pickle("P:/uni/nlp/WashingtonPost_v4/data/v5/"+str(c)+"_clean.pkl")
    
    for each in df2.Kategorie:
        kategorie.append(each)

    kategorie_all.append(list(kategorie))
    kategorie = []  


In [7]:
endlist = []
for each in kategorie_all:
    for each2 in each:
        endlist.append(each2)
        
endlist = list(set(endlist))

In [20]:
dfKat = pd.DataFrame(endlist)
#dfKat
dfKat = dfKat.sort_values(0)

In [24]:
dfKat.to_csv("test.csv")

In [25]:
dfkat_behalten = pd.read_csv("P:/uni/nlp/WashingtonPost_v4/Kategorie_behalten.CSV")

In [26]:
dfkat_behalten

,All Opinions Are Local
0,America Answers
1,Americas
2,Answer Sheet
3,Ask The Post
4,By The Way
...,...
61,The Washington Post
62,The Washington Post Magazine
63,The Watch
64,Think Tanked
